<a href="https://colab.research.google.com/github/Karthi-2-2/PROJECT/blob/main/Advanced_Time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
#Generate Data
import numpy as np
import pandas as pd

np.random.seed(42)
t = np.arange(1000)
data = 0.05*t + 10*np.sin(0.1*t) + np.random.normal(0, 1, 1000)
df = pd.DataFrame({"value": data})

#Create Sequences
def create_seq(data, window=30):
    X, y = [], []
    for i in range(len(data)-window):
        X.append(data[i:i+window])
        y.append(data[i+window])
    return np.array(X), np.array(y)

X, y = create_seq(df["value"].values)
X = X[..., np.newaxis]

#Transformer Model
import tensorflow as tf
from tensorflow.keras import layers, models

inputs = layers.Input(shape=(30,1))
x = layers.Dense(64)(inputs)

attn = layers.MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
x = layers.Add()([x, attn])
x = layers.LayerNormalization()(x)

x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1)(x)

model = models.Model(inputs, outputs)
model.compile(optimizer="adam", loss="mse")
model.summary()

#Train
model.fit(X, y, epochs=5, batch_size=32)

#Evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error

pred = model.predict(X)
mae = mean_absolute_error(y, pred)
rmse = np.sqrt(mean_squared_error(y, pred))
mape = np.mean(np.abs((y - pred.flatten()) / y)) * 100

print("MAE:", mae, "RMSE:", rmse, "MAPE:", mape)

#Baseline (ARIMA)
from statsmodels.tsa.arima.model import ARIMA

model_arima = ARIMA(df["value"], order=(2,1,2)).fit()
pred_arima = model_arima.predict(start=30, end=len(df)-1)

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 30, 1)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 30, 64)    │        128 │ input_layer_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 30, 64)    │     66,368 │ dense_8[0][0],    │
│ (MultiHeadAttentio… │                   │            │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 30, 64)    │          0 │ dense_8[0][0],    │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 30, 64)    │        128 │ add_4[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ layer_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 1)         │         65 │ global_average_p… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 66,689 (260.50 KB)

 Trainable params: 66,689 (260.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 570.0154
Epoch 2/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 395.3459
Epoch 3/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 334.9850
Epoch 4/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 295.4000
Epoch 5/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 264.4456
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
MAE: 12.310174465381078 RMSE: 14.955179213260303 MAPE: 265.3771513522424


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
